In [11]:
import os
from PIL import Image
import numpy
import sys

In [26]:
def create_training_set(root_train, transcription_file, train_file):
    training_dict = {}
    train = open(train_file).read().splitlines()
    lines = [line.rstrip('\n') for line in open(transcription_file)]
    for line in lines:
        split = line.split(' ')
        if split[0].split('-')[0] not in train:
            continue
        image = get_boolean_image(root_train + '/' + split[0] + '.jpg')
        if split[1] in training_dict:
            training_dict[split[1]].append(image)
        else:
            training_dict[split[1]] = []
            training_dict[split[1]].append(image)
    return training_dict

In [27]:
def get_boolean_image(file):
    # print("binarize %s" % (file_))
    im = Image.open(file).convert("L")
    height = im.size[0]
    width = im.size[1]
    px = im.load()
    for h in range(height):
        for w in range(width):
            if px[h, w] <= 125:
                px[h, w] = True
            else:
                px[h, w] = False
    return numpy.asarray(im)

In [28]:
training_dict = create_training_set('data/train','PatRec17_KWS_Data/ground-truth/transcription.txt','PatRec17_KWS_Data/task/train.txt')

In [109]:
print(training_dict.keys())

dict_keys(['s_2-s_7-s_0-s_pt', 'L-e-t-t-e-r-s-s_cm', 'O-r-d-e-r-s', 'a-n-d', 'I-n-s-t-r-u-c-t-i-o-n-s-s_pt', 'O-c-t-o-b-e-r', 's_1-s_7-s_5-s_5-s_pt', 'o-n-l-y', 'f-o-r', 't-h-e', 'p-u-b-l-i-c-k', 'u-s-e-s_cm', 'u-n-l-e-s_s-s', 'b-y', 'p-a-r-t-i-c-u-s_mi', 'l-a-r', 'f-r-o-m', 'm-e-s_pt', 'Y-o-u', 'a-r-e', 't-o', 's-e-n-d', 'd-o-w-n', 'a', 'B-a-r-r-e-l', 'o-f', 'F-l-i-n-t-s', 'w-i-t-h', 'A-r-m-s-s_cm', 'W-i-n-c-h-e-s-t-e-r-s_cm', 'a-b-o-u-t', 't-w-o', 't-h-o-u-s-a-n-d', 'w-e-i-g-h-t', 'F-l-o-u-r-s_cm', 'C-o-m-p-a-n-i-e-s', 'R-a-n-g-e-r-s-s_sq', 't-w-e-l-v-e', 'h-u-n-d-r-e-d', 'w-h-i-c-h', 'b-e', 'd-e-l-i-v-e-r-e-d', 'C-a-p-t-a-i-n', 'A-s-h-b-y', 'C-o-m-p-a-n-y-s_cm', 'a-t', 'P-l-a-n-t-a-t-i-o-n', 'C-h-a-r-l-e-s', 'S-e-l-l-a-r-s', 's_mi', 'r-e-s-t', 'C-o-c-k-e-s-s_qt', 'N-i-c-h-o-l-a-s', 'R-e-a-s-m-e-r-s-s_pt', 's_2-s_6th-s_pt', 's_GW', 's_2-s_8th', 'W-i-n-c-h-e-s-t-e-r-s_qo', 's_2-s_8th-s_cm', 'P-a-r-o-l-e', 'H-a-m-p-t-o-n-s_pt', 'T-h-e', 'O-f-f-i-c-e-r-s', 'w-h-o', 'c-a-m-e', 'F-o-r-t',

In [98]:
BLACK = True
WHITE = False


def calculate_features(training_dict):    
    return [calculate_image_features(image) for image in training_dict]


def calculate_image_features(image):
    windows = [image[:, i] for i in range(len(image[0]))]
    features = [calculate_window_features(window) for window in windows]
    return features


def calculate_window_features(window):
    feature_functions = (upper_contour, lower_contour, b_w_transitions, number_of_black_pixels, fraction_of_black_pixels_between_uc_and_lc, fraction_of_black_pixels)
    return [feature_function(window) for feature_function in feature_functions]


def upper_contour(window):
    for i in range(len(window)):
        if window[i] == BLACK:
            return i
    return -1


def lower_contour(window):
    for i in range(len(window)-1, -1, -1):
        if window[i] == BLACK:
            return i
    return -1


def b_w_transitions(window):
    pixel = window[0]
    transitions = 0
    for i in window:
        if i != pixel:
            transitions += 1
            pixel = i
    return transitions


def number_of_black_pixels(window):
    number_of_black = 0
    for i in range(len(window)):
        if window[i] == BLACK:
            number_of_black += 1
    return number_of_black

def fraction_of_black_pixels(window):
    return number_of_black_pixels(window) / len(window) * 100

def fraction_of_black_pixels_between_uc_and_lc(window):
    uc = upper_contour(window)
    lc = lower_contour(window)
    if uc == lc or uc > lc:
        return 0.0
    black_pixels_between_uc_and_lc = number_of_black_pixels(window[uc:lc+1])
    return (black_pixels_between_uc_and_lc / ((lc+1)-uc)) * 100


In [108]:
print("Features of the first image of an word 'Orders':")
calculate_features(training_dict['O-r-d-e-r-s'])[0]

Features of the first image of an word 'Orders':


[[-1, -1, 0, 0, 0.0, 0.0],
 [-1, -1, 0, 0, 0.0, 0.0],
 [-1, -1, 0, 0, 0.0, 0.0],
 [-1, -1, 0, 0, 0.0, 0.0],
 [-1, -1, 0, 0, 0.0, 0.0],
 [-1, -1, 0, 0, 0.0, 0.0],
 [-1, -1, 0, 0, 0.0, 0.0],
 [-1, -1, 0, 0, 0.0, 0.0],
 [-1, -1, 0, 0, 0.0, 0.0],
 [-1, -1, 0, 0, 0.0, 0.0],
 [49, 57, 2, 9, 100.0, 12.0],
 [47, 59, 2, 13, 100.0, 17.333333333333336],
 [44, 60, 2, 17, 100.0, 22.666666666666664],
 [42, 60, 2, 19, 100.0, 25.333333333333336],
 [41, 60, 4, 16, 80.0, 21.333333333333336],
 [39, 60, 4, 15, 68.18181818181817, 20.0],
 [39, 60, 4, 14, 63.63636363636363, 18.666666666666668],
 [37, 60, 4, 13, 54.166666666666664, 17.333333333333336],
 [35, 59, 4, 14, 56.00000000000001, 18.666666666666668],
 [33, 59, 4, 15, 55.55555555555556, 20.0],
 [32, 58, 4, 13, 48.148148148148145, 17.333333333333336],
 [31, 57, 4, 14, 51.85185185185185, 18.666666666666668],
 [29, 57, 6, 16, 55.172413793103445, 21.333333333333336],
 [28, 56, 6, 16, 55.172413793103445, 21.333333333333336],
 [27, 54, 6, 14, 50.0, 18.666666